In [7]:
from bs4 import BeautifulSoup
import requests
import traceback
from tqdm import tqdm
import json

In [39]:
# format of text file looks like:
"""
CSE         1244 (Summer 2024)         updated: 26-Feb-2024

                       class#    (autoenrolls)                                enrld/limit/+wait

     CSE 1110             5465 L                                      ONLINE       5/40       {8W1} M.Mallon

     CSE 1222             5466 L                 M W      0935A-1055  BE0188       5/40       {1S} L.Madrid
     CSE 1222             5467 B  ( 5466)           R     0935A-1055  BE0310       5/40       {1S} L.Madrid

     CSE 1223             8369 L                 M W      1245P-0205  CL0120       3/40       {1S} M.Mallon
     CSE 1223             8368 B  ( 8369)           R     1245P-0205  BE0470       3/40       {1S} M.Mallon
"""


'\nCSE         1244 (Summer 2024)         updated: 26-Feb-2024\n\n                       class#    (autoenrolls)                                enrld/limit/+wait\n\n     CSE 1110             5465 L                                      ONLINE       5/40       {8W1} M.Mallon\n\n     CSE 1222             5466 L                 M W      0935A-1055  BE0188       5/40       {1S} L.Madrid\n     CSE 1222             5467 B  ( 5466)           R     0935A-1055  BE0310       5/40       {1S} L.Madrid\n\n     CSE 1223             8369 L                 M W      1245P-0205  CL0120       3/40       {1S} M.Mallon\n     CSE 1223             8368 B  ( 8369)           R     1245P-0205  BE0470       3/40       {1S} M.Mallon\n'

In [40]:
# parse through the file. the first few lines are metadata and headers, then the rest are classes
# there are 8 columns: course info: {department, number}, course unique id: {section/class #}, {type} (e.g. L or B), {autoenrolls}, {days abbreviated as letters}, {time}, {location}, {enrolled/limit/+wait}, {instructor}
# completely separate classes are separated by a newline, two sections of the same class are separated in adjacent lines
# we want to extract the course info, section/class #, type, days, time, location, enrolled/limit/+wait, and instructor
# we want to output the data in a csv file
# the csv file should have the following columns: {department, number}, {section/class #}, {type}, {autoenrolls}, {days}, {time}, {location}, {enrolled}, {limit}, {+wait}, {instructor}
# the csv file should be named "classes.csv"

input_text_file = "1248.txt"
output_csv_file = "classes.csv"

with open(input_text_file, "r") as file:
    lines = file.readlines()
    # store first few lines as metadata
    metadata = lines[:3]
    # store the headers
    headers = lines[2].split()
    # store the classes from line 4 until we reach a line that begins with "INDependent study classes"
    classes = []
    for line in lines[4:]:
        if line.startswith("IND"):
            break
        classes.append(line)
    print(classes)

# remove leading and trailing whitespace
metadata = [line.strip() for line in metadata]
headers = [header.strip() for header in headers]
classes = [class_.strip() for class_ in classes]

print(metadata)
print(headers)
print(classes)

['     CSE 1110             6562 L                                      ONLINE       0/150      \n', '\n', '     CSE 1111             6563 L                  T R     0935A-1055  CL0220       0/40       \n', '     CSE 1111             6564 B  ( 6563)        M        1020A       DL0280       0/40       \n', '\n', '     CSE 1111             6565 L                  T R     1110A-1230  BE0140       0/40       \n', '     CSE 1111             6566 B  ( 6565)        M        1130A       DL0280       0/40       \n', '\n', '     CSE 1111             6567 L                  T R     0220P-0340  CL0220       0/40       \n', '     CSE 1111             6568 B  ( 6567)        M        0150P       DL0280       0/40       \n', '\n', '     CSE 1222             6930 L                  T R     0910A       BE0144       0/70       \n', '     CSE 1222             6931 B  ( 6930)        M        0800A       BE0480       0/40       \n', '     CSE 1222             7268 B  ( 6930)        M        0910A       BE04

In [41]:
# parse through the classes
parsed_classes = []
for class_ in classes:
    # split the class into its columns
    columns = class_.split()
    # if the columns are empty, then we have a new class
    if not columns:
        parsed_classes.append([])
    else:
        if len(parsed_classes) == 0:
            parsed_classes.append([])
        parsed_classes[-1].append(columns)
    
print(parsed_classes)

[[['CSE', '1110', '6562', 'L', 'ONLINE', '0/150']], [['CSE', '1111', '6563', 'L', 'T', 'R', '0935A-1055', 'CL0220', '0/40'], ['CSE', '1111', '6564', 'B', '(', '6563)', 'M', '1020A', 'DL0280', '0/40']], [['CSE', '1111', '6565', 'L', 'T', 'R', '1110A-1230', 'BE0140', '0/40'], ['CSE', '1111', '6566', 'B', '(', '6565)', 'M', '1130A', 'DL0280', '0/40']], [['CSE', '1111', '6567', 'L', 'T', 'R', '0220P-0340', 'CL0220', '0/40'], ['CSE', '1111', '6568', 'B', '(', '6567)', 'M', '0150P', 'DL0280', '0/40']], [['CSE', '1222', '6930', 'L', 'T', 'R', '0910A', 'BE0144', '0/70'], ['CSE', '1222', '6931', 'B', '(', '6930)', 'M', '0800A', 'BE0480', '0/40'], ['CSE', '1222', '7268', 'B', '(', '6930)', 'M', '0910A', 'BE0480', '0/40']], [['CSE', '1222', '6921', 'L', 'T', 'R', '1130A', 'DL0113', '0/160'], ['CSE', '1222', '6922', 'B', '(', '6921)', 'M', '1020A', 'BE0310', '0/40'], ['CSE', '1222', '10783', 'B', '(', '6921)', 'M', '1020A', 'BE0310', '0/40'], ['CSE', '1222', '7269', 'B', '(', '6921)', 'M', '1130A'

In [42]:
# parse through the parsed classes and continue to process the data to properly format it to be written to a csv file
# What I currently have: department, number, section/class #, type (e.g. L or B or others), autoenrolls (not always there), days (currently across multiple elements), time (not always present due to online classes), location, enrolled, limit, +wait (not always there), duration (not always present), instructor (not always present)
# I need to combine the days into a single element, and I need to account for the fact that some elements may not be present
# I also need to account for the fact that some classes have multiple sections

# I will create a list of dictionaries, where each dictionary represents a class
# I will then write the data to a csv file

# create a list of dictionaries
classes = []
for parsed_class in parsed_classes:
    course_info = []
    for class_ in parsed_class:
        class_info = {}
        class_copy = class_.copy()
        
        # line could begin with "and". if it does, add the entire line to the previous class_info dictionary as a "additional information" key
        if class_copy[0] == "and":
            class_copy.pop(0)
            course_info[-1]["additional information"] = " ".join(class_copy) # join the rest of the elements into a single string and add it to the dictionary
            continue

        # extract the department and number, then remove them from the list
        department, number = class_copy[0], class_copy[1]
        class_info["department"] = department
        class_info["number"] = number
        class_copy.pop(0)
        class_copy.pop(0)

        # extract the campus, section/class #, type, autoenrolls, days, time, location, enrolled, limit, +wait, duration, and instructor
        # campus is not always present, but if it is, it is the next element
        campus = ""
        if class_copy[0].isalpha():
            campus = class_copy[0]
            class_copy.pop(0)

        # section/class # and type are the next two elements
        section_class_number = class_copy[0]
        type_ = class_copy[1]
        class_copy.pop(0)
        class_copy.pop(0)

        # autoenrolls is not always present, but if it is, it is currently stored across two elements: element 4: "(", element 5: "{autoenrolls})" or element 4: "(autoenrolls)"
        # I want only the {autoenrolls} part
        autoenrolls = ""
        if class_copy[0].startswith("("):
            if class_copy[0] == "(":
                class_copy.pop(0)
                autoenrolls = class_copy[0].strip(")")
                class_copy.pop(0)
            else:
                autoenrolls = class_copy[0].strip("()")
                class_copy.pop(0)

        
        # days is currently stored across multiple elements, but might not be present (e.g. ONLINE classes do not have days or times)
        # combine the days into a single element (have to check if the days are present)
        days = ""
        # keep adding elements to days until we reach the time
        while class_copy[0][0] in ["M", "T", "W", "R", "F", "S", "U"]:
            days += class_copy[0]
            class_copy.pop(0)
        
        # time is not always present, but if it is, it starts with a digit
        time = ""
        if class_copy[0][0].isdigit() and "/" not in class_copy[0]:
            time = class_copy[0]
            class_copy.pop(0)
        
        # location is the next element. not always present, but if it is present, it is the next element
        location = ""
        if class_copy[0][0].isalpha():
            location = class_copy[0]
            class_copy.pop(0)

        # enrolled, limit, are stored across a single element as {enrolled}/{limit}. +wait is stored in the element after enrolled and limit, but +wait is not always present
        print(class_copy[0])
        enrolled, limit = class_copy[0].split("/")
        class_copy.pop(0)
        plus_wait = ""
        if len(class_copy) > 0 and class_copy[0].startswith("+"):
            plus_wait = class_copy[0]
            class_copy.pop(0)
        
        # duration is not always present, but if it is, it is the next element and startswith "{"
        duration = ""
        if len(class_copy) > 0 and class_copy[0].startswith("{"):
            duration = class_copy[0]
            class_copy.pop(0)
        
        # instructor is the next element, but is not always present
        instructor = ""
        # print(class_copy[0])
        if len(class_copy) > 0 and class_copy[0].isalpha():
            instructor = class_copy[0]
            class_copy.pop(0)
        
        # final descriptor of instructor is the next element, but is not always present (starts with "(")
        instructor_descriptor = ""
        if len(class_copy) > 0 and class_copy[0].startswith("("):
            instructor_descriptor = class_copy[0]
            class_copy.pop(0)
        
        # add the class to the list
        class_info["section/class #"] = section_class_number
        class_info["type"] = type_
        class_info["autoenrolls"] = autoenrolls
        class_info["days"] = days
        class_info["time"] = time
        class_info["location"] = location
        class_info["enrolled"] = enrolled
        class_info["limit"] = limit
        class_info["+wait"] = plus_wait
        class_info["duration"] = duration
        class_info["instructor"] = instructor
        class_info["instructor descriptor"] = instructor_descriptor
        course_info.append(class_info)
    classes.append(course_info)

print(classes)
        








0/150
0/40
0/40
0/40
0/40
0/40
0/40
0/70
0/40
0/40
0/160
0/40
0/40
0/40
0/40
0/160
0/40
0/40
0/40
0/40
0/160
0/40
0/40
0/40
0/40
0/160
0/40
0/40
0/40
0/40
0/160
0/80
0/80
0/160
0/80
0/80
0/160
0/80
0/80
0/160
0/80
0/80
0/160
0/80
0/80
0/160
0/80
0/80
0/160
0/80
0/80
0/160
0/80
0/80
0/160
0/80
0/80
0/160
0/80
0/80
0/160
0/80
0/80
0/160
0/80
0/80
0/160
0/80
0/80
0/160
0/80
0/80
0/80
0/80
0/40
0/0
0/0
0/0
0/0
0/0
0/0
0/0
0/0
0/0
0/0
0/151
0/80
0/40
0/40
0/0
0/40
0/40
0/40
0/40
0/40
0/40
0/40
0/40
0/40
0/40
0/40
0/40
0/40
0/40
0/40
0/40
0/40
0/40
0/40
0/40
0/40
0/40
0/40
0/40
0/40
0/40
0/40
0/40
0/40
0/40
0/40
0/40
0/40
0/40
0/40
0/40
0/40
0/40
0/40
0/40
0/40
0/40
0/40
0/40
0/40
0/40
0/40
0/40
0/40
0/40
0/40
0/40
0/40
0/40
0/34
0/40
0/40
0/40
0/32
0/40
0/40
0/40
0/40
0/40
0/40
0/40
0/40
0/40
0/40
0/40
0/40
0/40
0/40
0/40
0/40
0/45
0/45
0/45
0/60
0/60
0/45
0/45
0/40
0/45
0/45
0/45
0/45
0/45
0/45
0/45
0/45
0/45
0/45
0/45
0/60
0/60
0/60
0/45
0/45
0/45
0/45
0/40
0/40
0/30
0/30
0/30
0/30
0/30
0

In [43]:
# write the data to a json file
with open("testcourses-1248.json", "w") as f:
    json.dump(classes, f)

In [24]:
def split_class_file(content):
    lines = content.split("\n")
    # store first few lines as metadata
    # metadata goes from beginning of file to the line before the headers line which looks like "class#    (autoenrolls)                                enrld/limit/+wait"
    header_index = 0
    for i, line in enumerate(lines):
        if "class#" in line and "autoenrolls" in line and "enrld/limit/+wait" in line:
            header_index = i
            break
    metadata = lines[:header_index]
    # store the headers
    headers = lines[header_index].split()
    # store the classes from line header_index+2 until we reach a line that begins with "INDependent study classes" or "waitlist"
    classes = []
    for line in lines[header_index+2:]:
        if line.startswith("IND") or line.lower().startswith("waitlist"):
            # continue to pop the last element until we reach a non-empty line
            while len(classes) > 0 and len(classes[-1]) == 0:
                classes.pop()
            break
        classes.append(line)
    # print(classes)

    # remove leading and trailing whitespace
    metadata = [line.strip() for line in metadata]
    headers = [header.strip() for header in headers]
    classes = [class_.strip() for class_ in classes]

    return metadata, headers, classes

In [25]:
def preprocess_classes(classes):
    # parse through the classes
    parsed_classes = []
    for class_ in classes:
        # split the class into its columns
        columns = class_.split()
        # if the columns are empty, then we have a new class
        if not columns:
            parsed_classes.append([])
        else:
            if len(parsed_classes) == 0:
                parsed_classes.append([])
            parsed_classes[-1].append(columns)
        
    return parsed_classes

In [33]:
def parse_classes(parsed_classes):
    # parse through the parsed classes and continue to process the data to properly format it to be written to a csv file
    # What I currently have: department, number, section/class #, type (e.g. L or B or others), autoenrolls (not always there), days (currently across multiple elements), time (not always present due to online classes), location, enrolled, limit, +wait (not always there), duration (not always present), instructor (not always present)
    # I need to combine the days into a single element, and I need to account for the fact that some elements may not be present
    # I also need to account for the fact that some classes have multiple sections

    # I will create a list of dictionaries, where each dictionary represents a class
    # I will then write the data to a csv file

    # create a list of dictionaries
    
    try:
        classes = []
        for parsed_class in parsed_classes:
            course_info = []
            for class_ in parsed_class:
                class_info = {}
                class_copy = class_.copy()
                
                # line could begin with "and". if it does, add the entire line to the previous class_info dictionary as a "additional information" key
                if class_copy[0] == "and":
                    class_copy.pop(0)
                    course_info[-1]["additional information"] = " ".join(class_copy) # join the rest of the elements into a single string and add it to the dictionary
                    continue

                # extract the department and number, then remove them from the list
                department, number = class_copy[0], class_copy[1]
                class_info["department"] = department
                class_info["number"] = number
                class_copy.pop(0)
                class_copy.pop(0)

                # extract the campus, section/class #, type, autoenrolls, days, time, location, enrolled, limit, +wait, duration, and instructor
                # campus is not always present, but if it is, it is the next element
                campus = ""
                if class_copy[0].isalpha():
                    campus = class_copy[0]
                    class_copy.pop(0)

                # section/class # and type are the next two elements
                section_class_number = class_copy[0]
                type_ = class_copy[1]
                class_copy.pop(0)
                class_copy.pop(0)

                # autoenrolls is not always present, but if it is, it is currently stored across two elements: element 4: "(", element 5: "{autoenrolls})" or element 4: "(autoenrolls)"
                # I want only the {autoenrolls} part
                autoenrolls = ""
                if class_copy[0].startswith("("):
                    if class_copy[0] == "(":
                        class_copy.pop(0)
                        autoenrolls = class_copy[0].strip(")")
                        class_copy.pop(0)
                    else:
                        autoenrolls = class_copy[0].strip("()")
                        class_copy.pop(0)

                
                # days is currently stored across multiple elements, but might not be present (e.g. ONLINE classes do not have days or times)
                # combine the days into a single element (have to check if the days are present)
                days = ""
                # keep adding elements to days until we reach the time
                while class_copy[0][0].upper() in ["M", "T", "W", "R", "F", "S", "U"]:
                    days += class_copy[0]
                    class_copy.pop(0)
                
                # time is not always present, but if it is, it starts with a digit
                time = ""
                if class_copy[0][0].isdigit() and "/" not in class_copy[0]:
                    time = class_copy[0]
                    class_copy.pop(0)
                
                # location is the next element. not always present, but if it is present, it is the next element
                location = ""
                if class_copy[0][0].isalpha():
                    location = class_copy[0]
                    class_copy.pop(0)

                # enrolled, limit, are stored across a single element as {enrolled}/{limit}. +wait is stored in the element after enrolled and limit, but +wait is not always present
                # print(class_copy[0])
                enrolled, limit = class_copy[0].split("/")
                class_copy.pop(0)
                plus_wait = ""
                if len(class_copy) > 0 and class_copy[0].startswith("+"):
                    plus_wait = class_copy[0]
                    class_copy.pop(0)
                
                # duration is not always present, but if it is, it is the next element and startswith "{"
                duration = ""
                if len(class_copy) > 0 and class_copy[0].startswith("{"):
                    duration = class_copy[0]
                    class_copy.pop(0)
                
                # instructor is the next element, but is not always present
                instructors = []
                # print(class_copy[0])
                while len(class_copy) > 0 and class_copy[0][0].isalpha():
                    instructors.append(class_copy[0].strip(","))
                    class_copy.pop(0)
                
                # final descriptor of instructor is the next element, but is not always present (starts with "(")
                instructor_descriptor = ""
                if len(class_copy) > 0 and class_copy[0].startswith("("):
                    instructor_descriptor = class_copy[0]
                    class_copy.pop(0)
                
                # add the class to the list
                class_info["section/class #"] = section_class_number
                class_info["type"] = type_
                class_info["autoenrolls"] = autoenrolls
                class_info["days"] = days
                class_info["time"] = time
                class_info["location"] = location
                class_info["enrolled"] = enrolled
                class_info["limit"] = limit
                class_info["+wait"] = plus_wait
                class_info["duration"] = duration
                class_info["instructor"] = instructors
                class_info["instructor descriptor"] = instructor_descriptor
                course_info.append(class_info)
            classes.append(course_info)
    except Exception as e:
        # return the last class that was being processed
        return "Error", traceback.format_exc(), class_

    return "Success", "OK", classes

In [34]:
# prev_department_short_names = getDepartmentShortNames(1248)

prev_department_short_names_SP24 = ['ACADAFF', 'ACCTMIS', 'ACCAD', 'AEROENG', 'AFAMAST', 'AEDECON', 'ACEL', 'AGRCOMM', 'AGSYSMT', 'ASE', 'AIRSCI', 'AMINSTS', 'ASL', 'ANATOMY', 'ANIMSCI', 'ANMLTEC', 'ANTHROP', 'ARABIC', 'ARCH', 'ART', 'ARTEDUC', 'ARTSSCI', 'ASAMSTS', 'ASTRON', 'ATHTRNG', 'ATMOSSC', 'AVIATN', 'BIOCHEM', 'BIOETHC', 'BIOPHRM', 'BIOLOGY', 'BIOMEDE', 'BMI', 'BIOMSCI', 'BSGP', 'BIOPHYS', 'BIOTECH', 'BCS', 'BUSOBA', 'BUSFIN', 'BUSMHR', 'BUSML', 'BUSADM', 'BUSTEC', 'CONSRTM', 'CBG', 'CATALAN', 'CLLC', 'CHEMPHY', 'CBE', 'CHEM', 'CHINESE', 'CRPLAN', 'CIVILEN', 'CLAS', 'MCR', 'COMM', 'COMLDR', 'COMPSTD', 'CSE', 'CONSYSM', 'CSCFFS', 'CSFRST', 'CSHSPMG', 'CONSCI', 'CRPSOIL', 'CSTW', 'CZECH', 'DANCE', 'DENTHYG', 'DENT', 'DESIGN', 'DSABLST', 'DNE', 'EARTHSC', 'EALL', 'EEURLL', 'ECON', 'ESCE', 'ESCFE', 'ESEPSY', 'ESEADM', 'ESEPOL', 'ESHESA', 'ESLTECH', 'ESPHE', 'ESQUAL', 'ESQREM', 'ESSPSY', 'ESSPED', 'ESWDE', 'EHE', 'EDUTL', 'EDUCST', 'ECE', 'ENGR', 'ENGREDU', 'ENGTECH', 'ENGRTEC', 'ENGLISH', 'ENTMLGY', 'ENR', 'ENVENG', 'ENVSCI', 'ETHNSTD', 'EEOB', 'EXP', 'FILMSTD', 'FDSCTE', 'FABENG', 'FAES', 'FRENCH', 'FRIT', 'GENED', 'GENSTDS', 'GENBIOL', 'GENCHEM', 'GENCOMM', 'GENMATH', 'GENSSC', 'GEOSCIM', 'GEOG', 'GERMAN', 'GRADSCH', 'GREEK', 'HCINNOV', 'HIMS', 'HTHRHSC', 'HW', 'HWIH', 'HEBREW', 'HECCREG', 'HINDI', 'HISTORY', 'HISTART', 'HONORS', 'HORTTEC', 'HCS', 'HDFS', 'HUMNNTR', 'ISE', 'INTMED', 'INTSTDS', 'ISLAM', 'ITALIAN', 'JAPANSE', 'JEWSHST', 'KINESIO', 'KNHES', 'KNPE', 'KNSFHP', 'KNSISM', 'KNOW', 'KOREAN', 'LARCH', 'LATIN', 'LAW', 'LING', 'MBA', 'MDN', 'MATSCEN', 'MATH', 'MEATSCI', 'MECHENG', 'MEDDIET', 'MEDLBS', 'MEDCOLL', 'MEDREN', 'MEDMCIM', 'MICRBIO', 'MILSCI', 'MDRNGRK', 'MCDBIO', 'MOLGEN', 'MVNGIMG', 'MUSIC', 'NAVALSC', 'NELC', 'NEURSGY', 'NEURO', 'NEUROSC', 'NEUROGS', 'NUCLREN', 'NURSING', 'NRSADVN', 'NRSPRCT', 'OCCTHER', 'OSBP', 'OPTOM', 'OTOLARN', 'PATHOL', 'PEDS', 'PERSIAN', 'PHR', 'PHILOS', 'PHYSMED', 'PHYSTHR', 'PHYSICS', 'PHYSIO', 'PLNTPTH', 'POLISH', 'POLITSC', 'PORTGSE', 'PSYBHLH', 'PSYCH', 'PUBHBIO', 'PUBHEHS', 'PUBHEPI', 'PUBHHBP', 'PUBHHMP', 'PUBAFRS', 'PUBHLTH', 'QUECHUA', 'ROOM', 'RADONC', 'RADSCI', 'RADIOLG', 'RELSTDS', 'RESPTHR', 'ROMLING', 'ROMANIA', 'RURLSOC', 'RUSSIAN', 'SANSKRT', 'SCANDVN', 'SLAVIC', 'SOCWORK', 'SOCIOL', 'SOMALI', 'SASIA', 'SPANISH', 'SPHHRNG', 'STAT', 'SURGERY', 'SWAHILI', 'SWEDISH', 'TECPHYS', 'THEATRE', 'GRADTDA', 'TURKISH', 'URDU', 'UROLOGY', 'UZBEK', 'VETBIOS', 'VETCLIN', 'VMCOLL', 'VETPREV', 'VISSCI', 'WELDENG', 'WGSST', 'YIDDISH']
prev_department_short_names_AU24 = ['ACADAFF', 'ACCTMIS', 'ACCAD', 'AEROENG', 'AFAMAST', 'AEDECON', 'ACEL', 'AGRCOMM', 'AGSYSMT', 'ASE', 'AIRSCI', 'AMINSTS', 'ASL', 'ANATOMY', 'ANIMSCI', 'ANMLTEC', 'ANTHROP', 'ARABIC', 'ARCH', 'ART', 'ARTEDUC', 'ARTSSCI', 'ASAMSTS', 'ASTRON', 'ATHTRNG', 'ATMOSSC', 'AVIATN', 'BIOCHEM', 'BIOETHC', 'BIOPHRM', 'BIOLOGY', 'BMEA', 'BIOMEDE', 'BMI', 'BIOMSCI', 'BSGP', 'BIOPHYS', 'BIOTECH', 'BCS', 'BUSOBA', 'BUSFIN', 'BUSMHR', 'BUSML', 'BUSADM', 'BUSTEC', 'CONSRTM', 'CBG', 'CATALAN', 'CLLC', 'CHEMPHY', 'CBE', 'CHEM', 'CHINESE', 'CRPLAN', 'CIVILEN', 'CLAS', 'MCR', 'COMM', 'COMLDR', 'COMPSTD', 'CSE', 'CONSYSM', 'CSCFFS', 'CSFRST', 'CSHSPMG', 'CONSCI', 'CRPSOIL', 'CZECH', 'DANCE', 'DENTHYG', 'DENT', 'DESIGN', 'DSABLST', 'DNE', 'EARTHSC', 'EALL', 'ECON', 'ESCPSY', 'ESCE', 'ESCFE', 'ESEPSY', 'ESEADM', 'ESEPOL', 'ESHESA', 'ESLTECH', 'ESPHE', 'ESQUAL', 'ESQREM', 'ESSPSY', 'ESSPED', 'ESWDE', 'EHE', 'EDUTL', 'EDUCST', 'ECE', 'ENGR', 'ENGREDU', 'ENGTECH', 'ENGRTEC', 'ENGLISH', 'ENTMLGY', 'ENR', 'ENVENG', 'ENVSCI', 'ENVSCT', 'ETHNSTD', 'EEOB', 'EXP', 'FILMSTD', 'FDSCTE', 'FABENG', 'FAES', 'FRENCH', 'FRIT', 'GENED', 'GENSTDS', 'GENBIOL', 'GENCHEM', 'GENCOMM', 'GENMATH', 'GENSSC', 'GEOSCIM', 'GEOG', 'GERMAN', 'GRADSCH', 'GREEK', 'HCINNOV', 'HIMS', 'HTHRHSC', 'HW', 'HEBREW', 'HECCREG', 'HINDI', 'HISTORY', 'HISTART', 'HONORS', 'HORTTEC', 'HCS', 'HDFS', 'HUMNNTR', 'ISE', 'INTMED', 'INTSTDS', 'ISLAM', 'ITALIAN', 'JAPANSE', 'JEWSHST', 'KINESIO', 'KNHES', 'KNPE', 'KNSFHP', 'KNSISM', 'KNOW', 'KOREAN', 'LARCH', 'LATIN', 'LAW', 'LING', 'MBA', 'MDN', 'MATSCEN', 'MATH', 'MEATSCI', 'MECHENG', 'MEDDIET', 'MEDLBS', 'MEDCOLL', 'MEDREN', 'MEDMCIM', 'MICRBIO', 'MILSCI', 'MDRNGRK', 'MCDBIO', 'MOLGEN', 'MVNGIMG', 'MUSIC', 'NAVALSC', 'NELC', 'NEURSGY', 'NEURO', 'NEUROSC', 'NEUROGS', 'NUCLREN', 'NURSING', 'NRSADVN', 'NRSPRCT', 'OCCTHER', 'OSBP', 'OPTOM', 'OTOLARN', 'PATHOL', 'PEDS', 'PERSIAN', 'PHR', 'PHILOS', 'PHYSTHR', 'PHYSICS', 'PHYSIO', 'PLNTPTH', 'POLISH', 'POLITSC', 'PORTGSE', 'PSYBHLH', 'PSYCH', 'PUBHBIO', 'PUBHEHS', 'PUBHEPI', 'PUBHHBP', 'PUBHHMP', 'PUBAFRS', 'PUBHLTH', 'QUECHUA', 'ROOM', 'RADONC', 'RADSCI', 'RADIOLG', 'RELSTDS', 'RESPTHR', 'ROMANIA', 'RURLSOC', 'RUSSIAN', 'SANSKRT', 'SCANDVN', 'SCHOLAR', 'SLAVIC', 'SOCWORK', 'SOCIOL', 'SOMALI', 'SASIA', 'SPANISH', 'SPHHRNG', 'STAT', 'SURGERY', 'SWAHILI', 'SWEDISH', 'THEATRE', 'GRADTDA', 'TURKISH', 'URDU', 'UROLOGY', 'UZBEK', 'VETBIOS', 'VETCLIN', 'VMCOLL', 'VETPREV', 'VISSCI', 'WELDENG', 'WGSST', 'YIDDISH']
prev_department_short_names_SU24 = ['ACCTMIS', 'ACCAD', 'AEROENG', 'AFAMAST', 'AEDECON', 'ACEL', 'AGRCOMM', 'AGSYSMT', 'ASE', 'AMINSTS', 'ASL', 'ANATOMY', 'ANIMSCI', 'ANMLTEC', 'ANTHROP', 'ARABIC', 'ARCH', 'ART', 'ARTEDUC', 'ARTSSCI', 'ASAMSTS', 'ASTRON', 'ATHTRNG', 'ATMOSSC', 'AVIATN', 'BIOCHEM', 'BIOETHC', 'BIOPHRM', 'BIOLOGY', 'BIOMEDE', 'BMI', 'BSGP', 'BIOPHYS', 'BUSOBA', 'BUSFIN', 'BUSMHR', 'BUSML', 'BUSADM', 'BUSTEC', 'CONSRTM', 'CBG', 'CLLC', 'CHEMPHY', 'CBE', 'CHEM', 'CHINESE', 'CRPLAN', 'CIVILEN', 'CLAS', 'MCR', 'COMM', 'COMLDR', 'COMPSTD', 'CSE', 'CONSYSM', 'CSCFFS', 'CSFRST', 'CSHSPMG', 'CONSCI', 'CRPSOIL', 'DANCE', 'DENTHYG', 'DENT', 'DESIGN', 'DSABLST', 'DNE', 'EARTHSC', 'EALL', 'ECON', 'ESCE', 'ESEPSY', 'ESEADM', 'ESEPOL', 'ESHESA', 'ESLTECH', 'ESPHE', 'ESQUAL', 'ESQREM', 'ESSPSY', 'ESSPED', 'ESWDE', 'EHE', 'EDUTL', 'EDUCST', 'ECE', 'ENGR', 'ENGREDU', 'ENGTECH', 'ENGLISH', 'ENTMLGY', 'ENR', 'ENVENG', 'EEOB', 'EXP', 'FILMSTD', 'FDSCTE', 'FABENG', 'FAES', 'FRENCH', 'FRIT', 'GENED', 'GEOSCIM', 'GEOG', 'GERMAN', 'GRADSCH', 'GREEK', 'HCINNOV', 'HIMS', 'HTHRHSC', 'HW', 'HISTORY', 'HISTART', 'HORTTEC', 'HCS', 'HDFS', 'HUMNNTR', 'ISE', 'INTMED', 'INTSTDS', 'ITALIAN', 'JAPANSE', 'KINESIO', 'KNHES', 'KNPE', 'KNSFHP', 'KNSISM', 'KNOW', 'KOREAN', 'LARCH', 'LATIN', 'LAW', 'LING', 'MBA', 'MDN', 'MATSCEN', 'MATH', 'MEATSCI', 'MECHENG', 'MEDDIET', 'MEDLBS', 'MEDCOLL', 'MEDREN', 'MEDMCIM', 'MICRBIO', 'MCDBIO', 'MOLGEN', 'MVNGIMG', 'MUSIC', 'NELC', 'NEURSGY', 'NEURO', 'NEUROSC', 'NUCLREN', 'NURSING', 'NRSADVN', 'NRSPRCT', 'ORIENTAT', 'OCCTHER', 'OSBP', 'OPTOM', 'OTOLARN', 'PATHOL', 'PEDS', 'PERSIAN', 'PHR', 'PHILOS', 'PHYSTHR', 'PHYSICS', 'PHYSIO', 'PLNTPTH', 'POLITSC', 'PORTGSE', 'PSYBHLH', 'PSYCH', 'PUBHBIO', 'PUBHEHS', 'PUBHEPI', 'PUBHHBP', 'PUBHHMP', 'PUBAFRS', 'PUBHLTH', 'RADONC', 'RADSCI', 'RESPTHR', 'RURLSOC', 'RUSSIAN', 'SLAVIC', 'SOCWORK', 'SOCIOL', 'SPANISH', 'SPHHRNG', 'STAT', 'SURGERY', 'SWAHILI', 'THEATRE', 'GRADTDA', 'VETBIOS', 'VETCLIN', 'VMCOLL', 'VETPREV', 'VISSCI', 'WELDENG', 'WGSST']
prev_department_short_names_combined = set(prev_department_short_names_SP24 + prev_department_short_names_AU24 + prev_department_short_names_SU24) # eventually replace with department short names from database

# scrape through https://www.asc.ohio-state.edu/barrett.3/schedule/{department_short_name}/{term}.txt
dept_term_url = "https://www.asc.ohio-state.edu/barrett.3/schedule/{}/{}.txt"

# test_dept = ["CSE"] # replace later with prev_department_short_names_combined
# test_term = ["1248"] # replace later with [1242, 1244, 1248]

terms = [1242, 1244, 1248]
all_classes_by_term_department = {
    1242: {},
    1244: {},
    1248: {}
}
for dept in tqdm(prev_department_short_names_combined):
    for term in terms:
        print("Currently on department: {} and term: {}".format(dept, term))
        url = dept_term_url.format(dept, term)
        page = requests.get(url)
        # print(page.text)

        if page.status_code != 200:
            all_classes_by_term_department[term][dept] = []
            continue

        metadata, headers, classes = split_class_file(page.text)
        preprocessed_classes = preprocess_classes(classes)
        indicator, stacktrace, parsed_classes = parse_classes(preprocessed_classes)
        if indicator == "Error":
            print("Error occurred while parsing the file for department: {} and term: {}".format(dept, term))
            all_classes_by_term_department[term][dept] = {
                "error": stacktrace,
                "last_class": parsed_classes
            }
            continue
        else:
            all_classes_by_term_department[term][dept] = parsed_classes
        


# write the data from all_classes_by_term_department to a json file
with open("testclasses-ALL-1242-1244-1248.json", "w") as f: # search/fixtures/search/ is future location
    json.dump(all_classes_by_term_department, f)


  0%|          | 0/256 [00:00<?, ?it/s]

Currently on department: ENGTECH and term: 1242
Currently on department: ENGTECH and term: 1244


  0%|          | 1/256 [00:00<01:46,  2.40it/s]

Currently on department: ENGTECH and term: 1248
Currently on department: MEDCOLL and term: 1242
Currently on department: MEDCOLL and term: 1244
Currently on department: MEDCOLL and term: 1248


  1%|          | 2/256 [00:00<01:38,  2.58it/s]

...

Currently on department: PUBHBIO and term: 1248
Currently on department: HTHRHSC and term: 1242
Currently on department: HTHRHSC and term: 1244
Currently on department: HTHRHSC and term: 1248


100%|█████████▉| 255/256 [01:39<00:00,  2.45it/s]

Currently on department: ASTRON and term: 1242
Currently on department: ASTRON and term: 1244


100%|██████████| 256/256 [01:39<00:00,  2.56it/s]

Currently on department: ASTRON and term: 1248


In [28]:
# open the json file and read the data
with open("testclasses-ALL-1242-1244-1248.json", "r") as f:
    data = json.load(f)

for term in terms:
    for dept in prev_department_short_names_combined:
        if isinstance(data[str(term)][dept], list):
            continue
        else:
            print("Error occurred while parsing the file for department: {} and term: {}".format(dept, term))
            print(data[str(term)][dept]["error"])
            print(data[str(term)][dept]["last_class"])
            print()
            print()